In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import cv2

/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df=pd.read_csv('cifar10Labels.csv')

In [3]:
df.head()

,id,label
0,1,frog
1,2,truck
2,3,truck
3,4,deer
4,5,automobile


In [4]:
learning_rate=0.001
num_classes=10
num_steps=60000
batch_size=32

In [5]:
def conv_net(x_dict,n_classes):
    x=x_dict['images']
    conv1=tf.layers.conv2d(x,32,3,padding='same',)
    bn=tf.layers.batch_normalization(conv1)
    ac1=tf.nn.relu(bn)
    conv2=tf.layers.conv2d(ac1,32,3,padding='same',)
    bn2=tf.layers.batch_normalization(conv2)
    ac2=tf.nn.relu(bn2)
    maxpool1=tf.layers.max_pooling2d(ac2,pool_size=2,strides=2)
    conv3=tf.layers.conv2d(maxpool1,64,3,padding='same',)
    bn3=tf.layers.batch_normalization(conv3)
    ac3=tf.nn.relu(bn3)
    conv4=tf.layers.conv2d(ac3,64,3,padding='same',)
    bn4=tf.layers.batch_normalization(conv4)
    ac4=tf.nn.relu(bn4)
    maxpool2=tf.layers.max_pooling2d(ac4,pool_size=2,strides=2)
    flatten=tf.layers.flatten(maxpool2)
    fcn=tf.layers.dense(flatten,256)
    y_pred=tf.layers.dense(fcn,10)
    return y_pred

In [6]:
def model_fn(features,labels,mode):
    logits_train = conv_net(features, num_classes)
    logits_test = conv_net(features, num_classes)


    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)


    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)


    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op,
                                  global_step=tf.train.get_global_step())


    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)


    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [7]:
y_true=pd.get_dummies(df.label)

In [8]:
y_true=np.array(y_true)
y_true=np.argmax(y_true,axis=1)

In [9]:
y_true.shape

(50000,)

In [ ]:
data=[]
labels=[]
for image in glob.glob('train/*'):
    img=cv2.imread(image)
    name=image.split('/')[-1]
    print(name)
    index=str(name).split('.')[0]
    labels.append(y_true[int(index)-1])
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=img/255.
    data.append(img)
train=data[:48000]
test=data[48000:50000]
y_train=labels[:48000]
y_test=labels[48000:50000]
train=np.array(train)
test=np.array(test)
train=np.float32(train)
test=np.float32(test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [ ]:
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_secs = 2*60,  # Save checkpoints every 20 minutes.
    keep_checkpoint_max = 10       # Retain the 10 most recent checkpoints.
)
model = tf.estimator.Estimator(model_fn,model_dir='estimatormodel',config=my_checkpointing_config)

# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': train}, y=y_train,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

In [ ]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': test}, y=y_test,
    batch_size=32, shuffle=False)
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])
